In [1]:
from image_similarity import ImageSimilarity
from utils import copy_file_as_cluster

In [ ]:
# Usage
# folder_path = './data/YeonWooPinkBikini/'
folder_path = './data/blue_minidress/'
s = ImageSimilarity(folder_path,
                    model_name='OFA-Sys/chinese-clip-vit-huge-patch14',
                    batch_size=16,
                    show_progress_bar=True)

Initializing ImageSimilarity...


2024-01-06 12:32:23.777 | DEBUG    | similarities.clip_module:__init__:58 - Device: cpu


In [1]:
# clusters = s.cluster_images_with_hierarchical(distance_threshold=0.5)
clusters = s.cluster_images_with_multilevel_hierarchical(distance_levels=[2, 0.5])
pprint.pprint(clusters)

NameError: name 's' is not defined

In [13]:

# Example usage
# Assuming 'clusters' is your multi-level hierarchical cluster dictionary
copy_files_as_cluster(clusters, './data/testoutput/')



## test

In [53]:
questions = {
    "explicit_content": "Does this photo contain explicit content?",
    "cloth_color": "which color or colors or nude does the girl wear?",
    "camera_angle": "which angle does the photo take from according to the girl?",
    "pose": "which pose does the girl perform?",
    "is_sex": "is the main character in the picture having sex currently",
    "is_blowjob": "is it doing a oral sex?",
    "is_doggy": "is it having sex in doggy style from behind?",
    "is_virginia_penetration": "is there virginia penetrated by penis?",
    "pose_hand": "what is the pose of her hand",
    "pose_thigh": "what is the pose of her thigh",
    "pose_crotch": "what is the pose of her crotch",
    "pose_leg": "what is the pose of her leg",
    "name": "give a informative filename for this photo that make it unique",
    # Add other keywords and questions as needed
}

In [57]:
# Usage example
# qa = ImageQuestionAnswerer("blip_vqa", "vqav2")
qa = ImageQuestionAnswerer("blip_caption", "large_coco")

# qa = ImageQuestionAnswerer("blip2_opt", "caption_coco_opt2.7b")

In [50]:
# img = s.thumbnail_cache_manager.load('./data/blue_minidress/IMG_2050.HEIC')
# img = s.thumbnail_cache_manager.load('./data/blue_minidress/IMG_2661.HEIC')
# img = s.thumbnail_cache_manager.load('./data/blue_minidress/IMG_2066.HEIC')
# img = s.thumbnail_cache_manager.load('./data/blue_minidress/IMG_2840.HEIC')
# img = s.thumbnail_cache_manager.load('./data/blue_minidress/IMG_2726.HEIC')
img = s.thumbnail_cache_manager.load('./data/blue_minidress/IMG_2077.MOV')
# img = s.thumbnail_cache_manager.load('./data/blue_minidress/IMG_2076.MOV')
# img = s.thumbnail_cache_manager.load('./data/blue_minidress/IMG_2007.HEIC')

In [55]:
for fp in s.media_fps:
    res = qa.caption(s.thumbnail_cache_manager.load(fp))
    print(fp, res)

./data/blue_minidress/IMG_2007.HEIC ['a woman in a gray dress standing in a living room']
./data/blue_minidress/IMG_2008.HEIC ['a woman sitting on a window sill in front of a window']
./data/blue_minidress/IMG_2009.HEIC ['a woman sitting on a window sill in front of a window']
./data/blue_minidress/IMG_2010.HEIC ['a woman is sitting on a window sill']
./data/blue_minidress/IMG_2011.HEIC ['a woman is sitting on a window sill']
./data/blue_minidress/IMG_2012.HEIC ['a woman sitting on a window sill in a room']
./data/blue_minidress/IMG_2013.HEIC ['a woman is sitting on a window sill']
./data/blue_minidress/IMG_2014.HEIC ['a woman is sitting on a window sill']


KeyboardInterrupt: 

In [51]:
%%time
for k, v in questions.items():
    res = qa.ask(img, v)
    print(v, res)

Does this photo contain explicit content? ['yes']
which color or colors or nude does the girl wear? ['gray']
which angle does the photo take from according to the girl? ['straight']
which pose does the girl perform? ['back']
is the main character in the picture having sex currently ['yes']
is it doing a oral sex? ['no']
is it having sex in doggy style from behind? ['yes']
is there virginia penetrated by penis? ['yes']
what is the pose of her hand ['on her butt']
what is the pose of her thigh ['bent']
what is the pose of her crotch ['bent']
what is the pose of her leg ['bent']
give a informative filename for this photo that make it unique ['naked woman']
CPU times: user 40.6 s, sys: 3.32 s, total: 43.9 s
Wall time: 41 s


In [63]:
qa.rank(img, 'which angle is the photo taken from based on the girl', 
        [
            'Front (0 degree)',
            'Side Profile (90 Degrees)',
            'Back (180 Degrees)',
            'Slight Side Back (150 Degrees)',
            'Three-Quarter View (45 Degrees)',
            'Slight Side Angle (15-30 Degrees)',
        ])

['Back (180 Degrees)']

In [66]:
qa.rank(img, 'which angle is the photo taken from based on the girl', 
        [
            "eye level",
            "high angle",
            "low angle",
            "bird's eye view",
            "worm's eye view",
            "over the shoulder shot",
            "close up",
            "frog view",
        ])

['low angle']